## Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
import json

import sys
sys.path.insert(0, '..')

In [3]:
import os

from tqdm.notebook import tqdm
import pandas as pd
from llama_index.core.response.notebook_utils import display_source_node

In [4]:
import nest_asyncio 

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

## Load Config

In [6]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Healthcare bot",
    RUN_NAME="03_chatbot_ui",
    RUN_DESCRIPTION="""
# Objective
Create ui chatbot
    """,
    TESTING=False,
    DEBUG=False,
    OBSERVABILITY=True,
    LOG_TO_MLFLOW=True,
    CREATE_RETRIEVAL_EVAL_DATASET=False,
    RECREATE_RESPONSE_EVAL_DATASET=False,
    RECREATE_INDEX=False,
)
ARGS

{
  "EXPERIMENT_NAME": "Healthcare bot",
  "RUN_NAME": "03_chatbot_ui",
  "RUN_DESCRIPTION": "\n# Objective\nCreate ui chatbot\n    ",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "CREATE_RETRIEVAL_EVAL_DATASET": false,
  "RECREATE_RESPONSE_EVAL_DATASET": false,
  "RECREATE_INDEX": false
}

In [7]:
from src.run.cfg import RunConfig

cfg = RunConfig()
cfg.init(ARGS)

2024-09-24 22:54:56.340 | INFO     | src.run.cfg:init:136 - Starting Observability server with Phoenix...
INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/nampq/.phoenix
E0000 00:00:1727193302.086376   20399 chttp2_server.cc:1118] UNKNOWN:No address added out of total 1 resolved for '[::]:4317' {created_time:"2024-09-24T22:55:02.086360521+07:00", children:[UNKNOWN:Failed to add any wildcard listeners {created_time:"2024-09-24T22:55:02.086350318+07:00", children:[UNKNOWN:Unable to configure socket {fd:65, created_time:"2024-09-24T22:55:02.086323254+07:00", children:[UNKNOWN:bind: Address already in use (98) {created_time:"2024-09-24T22:55:02.08630348+07:00"}]}, UNKNOWN:Unable to configure socket {fd:65, created_time:"2024-09-24T22:55:02.086347781+07:00", children:[UNKNOWN:bind: Address already in use (98) {created_time:"2024-09-24T22:55:02.086344578+07:00"}]}]}]}
ERROR:    Traceback (most recent call last):
  File "/home/nampq/.cache/pypoetry/virtualenvs/personal-bot-bBSb

In [8]:
from llama_index.llms.gemini import Gemini

In [9]:
llm = Gemini(
    model_name="models/gemini-1.5-flash",
    temperature=0.3
)

In [10]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool

In [11]:
# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [12]:
agent = ReActAgent.from_tools(tools=[multiply_tool], llm=llm, verbose=True)

In [13]:
agent.chat("2023 * 2025 bằng mấy?")

> Running step 7b2392d0-dd9f-4ecc-ae81-1818fe5f91be. Step input: 2023 * 2025 bằng mấy?
Thought: The current language of the user is: Vietnamese. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2023, 'b': 2025}
Observation: 4096575
> Running step 9a29b971-98d1-464e-be51-fe379916fd0d. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 2023 * 2025 bằng 4096575.


AgentChatResponse(response='2023 * 2025 bằng 4096575.', sources=[ToolOutput(content='4096575', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2023, 'b': 2025}}, raw_output=4096575, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [16]:
res = agent.chat("2023 * 2025 bằng mấy?")

> Running step 781c2f86-b212-4bef-a8cb-00ef41d51304. Step input: 2023 * 2025 bằng mấy?
Thought: I need to use a tool to calculate the product.
Action: multiply
Action Input: {'a': 2023, 'b': 2025}
Observation: 4096575
> Running step 96aa1e56-2c22-4a57-b098-31409208fd84. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 2023 * 2025 bằng 4096575.


In [17]:
res.response

'2023 * 2025 bằng 4096575.'